In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def probiou_loss(boxes_pred, target_boxes_, EPS = 1e-3, mode='l2'):

    """
        boxes_pred    -> a matrix [N,5](x,y,w,h,angle - in radians) containing ours predicted box ;in case of HBB angle == 0
        target_boxes_  -> a matrix [N,5](x,y,w,h,angle - in radians) containing ours target    box ;in case of HBB angle == 0
        EPS     -> threshold to avoid infinite values
        mode       -> ('l1' in [0,1] or 'l2' in [0,inf]) metrics according our paper

    """

    x1, y1, w1, h1, theta1 = tf.unstack(boxes_pred, axis=1)
    x2, y2, w2, h2, theta2 = tf.unstack(target_boxes_, axis=1)
    x1 = tf.reshape(x1, [-1, 1])
    y1 = tf.reshape(y1, [-1, 1])
    h1 = tf.reshape(h1, [-1, 1])
    w1 = tf.reshape(w1, [-1, 1])
    theta1 = tf.reshape(theta1, [-1, 1])
    x2 = tf.reshape(x2, [-1, 1])
    y2 = tf.reshape(y2, [-1, 1])
    h2 = tf.reshape(h2, [-1, 1])
    w2 = tf.reshape(w2, [-1, 1])
    theta2 = tf.reshape(theta2, [-1, 1])

    # gbb form
    aa = w1**2/12; bb = h1**2/12; angles = theta1
    # rotated form
    a1 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
    b1 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
    c1 = 0.5*(aa - bb)*tf.math.sin(2.*angles)

    # gbb form
    aa = w2**2/12; bb = h2**2/12; angles = theta2
    # rotated form
    a2 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
    b2 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
    c2 = 0.5*(aa - bb)*tf.math.sin(2.*angles)

    # Formulação matricial
    sigma1 = tf.constant([[a1, c1],
                          [c1, b1]], dtype=tf.float32)

    sigma2 = tf.constant([[a2, c2],
                          [c2, b2]], dtype=tf.float32)

    mu1 = tf.constant([[x1],
                       [y1]], dtype=tf.float32)

    mu2 = tf.constant([[x2],
                       [y2]], dtype=tf.float32)
    